In [4]:
import json
import pandas as pd

file_path = 'example_result.jsonl'

records = []
with open(file_path, 'r') as f:
    buffer = []
    depth = 0

    for raw in f:
        line = raw.strip()
        if not line:
            continue  # skip blank lines

        # Track nesting of braces
        depth += line.count('{')
        depth -= line.count('}')

        buffer.append(raw)
        # If we’ve closed all opened braces, process one full JSON object
        if depth == 0 and buffer:
            chunk = ''.join(buffer)
            obj = json.loads(chunk)
            # same flattening logic as before
            for uid, details in obj.items():
                params = details.pop('params')
                flat = {'id': uid, **details}
                for p, v in params.items():
                    if isinstance(v, list) and len(v) == 2:
                        flat[f'{p}_min'] = v[0]
                        flat[f'{p}_max'] = v[1]
                    else:
                        flat[p] = v
                records.append(flat)
            buffer = []

# Build DataFrame
df = pd.DataFrame(records)
print(df)


                                       id         score  total_ghosts  \
0    01f75fbe-80ce-42fc-bf4b-84f8be9d13ee  16819.240810          5873   
1    35762ce3-fa82-44ab-b676-a8f3cf79cd6b  15977.776768          5604   
2    d280c3c2-5957-431d-bc2b-d136535c5b4d  16976.487219          6097   
3    42839014-0de9-44b7-b334-b922947940ae  15112.041044          5158   
4    dcd22691-c4de-4353-b077-4fe3201084e6  16130.521219          5815   
..                                    ...           ...           ...   
995  cfc029f0-59a4-4b68-ad66-5ce8e6330d6a     47.750969             7   
996  03fcc834-b04f-47fd-8613-c408a5ca7a5e     47.750870             7   
997  5adc6a5d-f309-4185-b2a0-4ffa6324322e     47.750833             7   
998  24b77033-8fa4-4f16-bb4b-1eae5619454f     47.750808             7   
999  328644f9-ec84-4b73-bce9-c258fcaed659     47.750798             7   

     overall_ghost_rate  event_avg_ghost_rate  clone_percentage  \
0             17.468249             15.150894          5

In [5]:
df.head()

,id,score,total_ghosts,overall_ghost_rate,event_avg_ghost_rate,clone_percentage,max_slope_min,max_slope_max,max_tol_min,max_tol_max,scatter
0,01f75fbe-80ce-42fc-bf4b-84f8be9d13ee,16819.240810,5873,17.468249,15.150894,5.708007,0.50,0.50,0.50,0.50,0.4
1,35762ce3-fa82-44ab-b676-a8f3cf79cd6b,15977.776768,5604,17.546496,15.174806,5.790740,0.25,0.50,0.25,0.50,0.4
2,d280c3c2-5957-431d-bc2b-d136535c5b4d,16976.487219,6097,17.966701,15.570292,5.706144,0.75,0.50,0.75,0.50,0.4
3,42839014-0de9-44b7-b334-b922947940ae,15112.041044,5158,17.075512,14.802821,5.928304,0.25,0.25,0.25,0.25,0.4
4,dcd22691-c4de-4353-b077-4fe3201084e6,16130.521219,5815,18.034922,15.609491,5.788198,0.25,0.75,0.25,0.75,0.4
